In [348]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model  # will be using for plotting trend line
from sklearn.preprocessing import MinMaxScaler # for normalizing data
from sklearn.cluster import KMeans 
%matplotlib inline

In [366]:
spotify = pd.read_csv('archive/data.csv')
spotify.head(1) ##maybe remove everything before 1946 

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.195,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.151,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.779,1928


In [350]:
##scrape year end top 100 songs from wiki
from html.parser import HTMLParser
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
import re

In [351]:
def convert(s): 
    # initialization of string to "" 
    str1 = "" 
  
    # using join function join the list s by  
    # separating words by str1 
    return(str1.join(s)) 


In [352]:
url = 'https://en.wikipedia.org/wiki/Billboard_year-end_top_singles_of_1946'
response = urllib.request.urlopen(url)
soup = BeautifulSoup(response, 'html.parser')
links = ["https://en.wikipedia.org/wiki/Billboard_year-end_top_singles_of_1946"]
for l in soup.find_all('a'):
#     print(l)
    if l.has_attr('title'):
        if((l.attrs['title'][0:19] =="Billboard Year-End ") or (l.attrs['title'][0:19] =="Billboard year-end ")):
#             print(l.attrs['href'])
            link = "https://en.wikipedia.org" + l.attrs['href']
            links.append(link)

In [353]:
all_billboard_df = pd.DataFrame(columns=["Rank", "Title", "Artist"])
for link in links:
    url = link
#     print(url)
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response, 'html.parser')
    names = []
    for l in soup.find_all('td'):
        names.append(l.text)
    mylist = []
    for name in names:
        mylist.append(name.strip().split('\n')) #remove that
    #remove any item in the list longer thene len 1
    new_list=[x for x in mylist if len(x)<2]
    for idx, val in enumerate(new_list[::3]):  ##cute anything in the list that is not a rank
        if (str.isdigit(convert(val)) == True):
            check = "good"
        else:
            cut = idx
    new_list = new_list[:(cut*3)]
#             print(new_list)
    rank = new_list[::3] # get evert 3rd item
    title = new_list[1::3]
    artist = new_list[2::3] 
    this_billboard_df = pd.DataFrame(columns=["Rank", "Title", "Artist"])
    this_billboard_df["Rank"] = rank
    this_billboard_df["Title"] = title
    this_billboard_df["Artist"] = artist
    for i, row in this_billboard_df.iterrows():
        this_billboard_df.set_value(i,'Rank',convert(row[0]))
        this_billboard_df.set_value(i,'Title',convert(row[1])) 
        this_billboard_df.set_value(i,'Title',row[1].strip('"'))
        this_billboard_df.set_value(i,'Artist',convert(row[2]))
    all_billboard_df = pd.concat([all_billboard_df, this_billboard_df])



/Users/adiepresent/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/adiepresent/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/adiepresent/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/adiepresent/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [411]:
all_billboard_df

,Rank,Title,Artist
0,1,Prisoner of Love,Perry Como
1,2,To Each His Own,Eddy Howard
2,3,The Gypsy,Ink Spots
3,4,Five Minutes More,Frank Sinatra
4,5,Rumors Are Flying,Frankie Carle
5,6,Oh! What It Seemed to Be,Frankie Carle
6,7,Personality,Johnny Mercer and The Pied Pipers
7,8,"South America, Take It Away",Bing Crosby and The Andrews Sisters
8,9,The Gypsy,Dinah Shore
9,10,Oh! What It Seemed to Be,Frank Sinatra


In [355]:
all_billboard_df.to_csv('billboard_data.csv', index=False,)

In [368]:
#assign an indicator next for each song to indicate if that eexact song was ever in the top100
spotify = spotify.assign(Top100=spotify.name.isin(all_billboard_df.Title).astype(int)) 


##maybe loosen this to see if its not matching songs when just the capitalization or something stupid is off


#double merge song and artist


In [370]:
counts = spotify[['Top100', "name"]].rename(columns={"name": "count"}).groupby("Top100").count()
counts

,count
Top100,
0,158320
1,11589


In [371]:
#data frame of just the songs that are top100 songs -- could be useful
popular = spotify.loc[spotify['Top100'] == 1]
popular.tail()


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,Top100
169750,0.0557,['AJR'],0.528,245270,0.727,0,3VygfAvvgVaJUeaBUSzlZu,0.000000,6,0.1760,-5.739,1,Karma,63,2019-04-26,0.1790,155.927,0.2120,2019,1
169762,0.0290,"['Shoreline Mafia', 'OHGEESY', 'Fenix Flexin']",0.927,144080,0.607,1,0MdSezTbC54SIPQkXrvHrp,0.000000,4,0.1040,-5.372,0,Wings,65,2019-09-04,0.1900,101.999,0.3640,2019,1
169853,0.0946,['Joel Corry'],0.697,190955,0.921,0,2ZEq4HT450Ye9IFGPTl9qV,0.000019,5,0.1590,-4.283,1,Lonely,81,2020-01-24,0.0424,123.988,0.7220,2020,1
169872,0.1060,['Sean Paul'],0.951,218573,0.600,0,6QvolntlpSFucqR9f91AoK,0.000000,0,0.0712,-4.675,0,Temperature,1,2020-06-12,0.0686,125.040,0.8220,2020,1
169893,0.3040,"['Kygo', 'Patrick Droney', 'Petey']",0.593,207898,0.430,0,5vDjcNbN4m9fxWcrpR64Wu,0.000000,0,0.0896,-8.977,1,Say You Will,69,2020-05-29,0.0431,122.090,0.0955,2020,1


In [405]:
#convert the artists in spotify data to a form that cen be compared to billboard data
for i, row in spotify.iterrows():
    s = row[1].replace(',', ' and')
    s = s.strip("]")
    s = s.strip("[")
    s = s.strip("'")
    spotify.set_value(i,'artists',s)


/Users/adiepresent/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [407]:
#add an indicator for every song to indicate if that artist has ever had a song in the top100, even if this song may not be a hit
#this column will be good for the part where we coontrol for popular artists and try to figure out why only some of their songs are "hits"

spotify = spotify.assign(pop_artist=spotify.artists.isin(all_billboard_df.Artist).astype(int)) 


In [409]:
#count the number of songs in spotify that come from a top100 artist
counts_1 = spotify[['pop_artist', "name"]].rename(columns={"name": "count"}).groupby("pop_artist").count()
counts_1


,count
pop_artist,
0,125009
1,44900


In [410]:
spotify.to_csv('spotify_billboard_data.csv', index=False,)

In [412]:
spotify

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,...,mode,name,popularity,release_date,speechiness,tempo,valence,year,Top100,pop_artist
0,0.9950,Carl Woitschach,0.708,158648,0.19500,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563000,10,0.1510,...,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928,0,0
1,0.9940,Robert Schumann' and 'Vladimir Horowitz,0.379,282133,0.01350,0,6KuQTIu1KoTTkLXKrwlLPV,0.901000,8,0.0763,...,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928,0,0
2,0.6040,Seweryn Goszczyński,0.749,104300,0.22000,0,6L63VW0PibdM1HDSBoqnoM,0.000000,5,0.1190,...,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928,0,0
3,0.9950,Francisco Canaro,0.781,180760,0.13000,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,...,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928,0,0
4,0.9900,Frédéric Chopin' and 'Vladimir Horowitz,0.210,687733,0.20400,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908000,11,0.0980,...,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928,0,0
5,0.9950,Felix Mendelssohn' and 'Vladimir Horowitz,0.424,352600,0.12000,0,6NxAf7M8DNHOBTmEd3JSO5,0.911000,6,0.0915,...,0,Scherzo a capriccio: Presto,0,1928,0.0593,63.521,0.2660,1928,0,0
6,0.9560,Franz Liszt' and 'Vladimir Horowitz,0.444,136627,0.19700,0,6O0puPuyrxPjDTHDUgsWI7,0.435000,11,0.0744,...,1,"Valse oubliée No. 1 in F-Sharp Major, S. 215/1",0,1928,0.0400,80.495,0.3050,1928,0,0
7,0.9880,Carl Woitschach,0.555,153967,0.42100,0,6OJjveoYwJdIt76y0Pxpxw,0.836000,1,0.1050,...,1,Per aspera ad astra,0,1928,0.0474,123.310,0.8570,1928,0,0
8,0.9950,Francisco Canaro' and 'Charlo,0.683,162493,0.20700,0,6OaJ8Bh7lsBeYoBmwmo2nh,0.206000,9,0.3370,...,0,Moneda Corriente - Remasterizado,0,1928-10-03,0.1270,119.833,0.4930,1928,0,0
9,0.8460,Seweryn Goszczyński,0.674,111600,0.20500,0,6PrZexNb16cabXR8Q418Xc,0.000000,9,0.1700,...,1,Chapter 1.3 - Zamek kaniowski,0,1928,0.9540,81.249,0.7590,1928,0,0
